In [26]:
import numpy as np
import pandas as pd

import tensorflow as tf

#from fastDamerauLevenshtein import damerauLevenshtein
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split
from tensorflow.keras import layers

In [53]:
df = pd.read_csv('data/single_step_df_ints_2022-04-13_categorized.csv')

In [54]:
df

,input,target,start_coords.x,start_coords.y,start_coords.z,tray.already_seen,bowl.already_seen,wineglass.already_seen,silverware.already_seen,spoon.already_seen,...,cookware.food_k,cookware.mid_k,cookware.strong_k,cookware.x,cookware.y,cookware.z,cookware.already_seen,bowl.x,bowl.y,bowl.z
0,<start>,plate,-0.451354,-0.413918,0.156247,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,plate,plate_small,0.513000,-0.531000,0.740000,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,plate_small,cup,0.513000,-0.531000,0.740000,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,cup,glass,0.513000,-0.531000,0.740000,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,glass,knife,0.513000,-0.531000,0.740000,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2096,<start>,food,1.000000,0.000000,2.000000,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2097,pineapple,cutting_board,0.000000,1.500000,2.000000,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2098,cutting_board,knife,0.000000,1.500000,2.000000,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2099,knife,container,0.000000,1.500000,2.000000,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [55]:
float_cols = df.select_dtypes(include=['float64']).columns
df.loc[:, float_cols] = df.loc[:, float_cols].fillna(-99)

In [56]:
df['target'].unique()
df['target'].value_counts()

knife            279
plate            249
fork             172
glass            172
food             169
cup              152
spoon            146
bowl             114
cutting_board    107
wineglass         74
bottle            74
plate_small       73
utensil           56
spoon_small       47
seasoning         43
drink             36
cookware          30
fork_small        22
tray              22
silverware        21
teaspoon          13
placemat          13
napkin             9
container          7
egg_cup            1
Name: target, dtype: int64

In [57]:
index_to_drop = df['target'].loc[df['target'] == 'egg_cup'].index[0]
index_to_drop

1972

In [58]:
nr_of_sequences = len(df[df['input'] == '<start>'])

In [59]:
# remove row with target that only occurs once

df_new = df.drop(index=index_to_drop, axis=0)
df_new.reset_index(inplace=True, drop=True)

In [60]:
# generate stratified split

split = StratifiedShuffleSplit(n_splits=10, test_size=0.3, random_state=42)

for train_index, test_index in split.split(df_new, df_new['target']):
    strat_train = df_new.loc[train_index]
    strat_test_val = df_new.loc[test_index]

In [61]:
# split test_val into test and val

split_test_val = len(strat_test_val)//2

strat_test = strat_test_val[:split_test_val]
strat_val = strat_test_val[split_test_val:]

In [62]:
# generate list of labels to pass to MultiLabelBinarizer so there's the same number of
# classes for all datasets

labels = df_new['target'].unique()

In [63]:
def create_dataset(dataframe, labels, shuffle=True, batch_size=32):
    df = dataframe.copy()
    labels_ds = df.pop('target')
    encoder = MultiLabelBinarizer(classes=labels)
    encoded_labels = encoder.fit_transform(labels_ds)
    
    df = {key: value[:, tf.newaxis] for key, value in df.items()}
    dataset = tf.data.Dataset.from_tensor_slices((dict(df), encoded_labels))
    if shuffle:
        dataset = dataset.shuffle(buffer_size=len(dataframe))
    
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(batch_size)
    
    return dataset

In [64]:
batch_size = 128

In [65]:
train_ds = create_dataset(strat_train, labels, batch_size=batch_size)
val_ds = create_dataset(strat_val, labels, shuffle=False, batch_size=batch_size)
test_ds = create_dataset(strat_test, labels, shuffle=False, batch_size=batch_size)

/home/zauri/files/miniconda3/envs/tf2.7/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:875: UserWarning: unknown class(es) ['_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'i', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'y'] will be ignored
  warnings.warn(
/tmp/ipykernel_29915/1510453973.py:7: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  df = {key: value[:, tf.newaxis] for key, value in df.items()}


In [66]:
def get_normalization_layer(name, dataset):
  # Create a Normalization layer for the feature.
    normalizer = layers.Normalization(axis=None)

  # Prepare a Dataset that only yields the feature.
    feature_ds = dataset.map(lambda x, y: x[name])

  # Learn the statistics of the data.
    normalizer.adapt(feature_ds)
    
    return normalizer

In [67]:
def get_category_encoding_layer(name, dataset, dtype, max_tokens=None):
    if dtype == 'string':
        index = layers.StringLookup(max_tokens=max_tokens)
    else:
        index = layers.IntegerLookup(max_tokens=max_tokens)
        
    # prepare tf.data.Dataset that only yields the feature    
    feature_ds = dataset.map(lambda x, y: x[name])
    
    # learn set of possible values and assign fixed int index
    index.adapt(feature_ds)
    
    # encode int indices
    encoder = layers.CategoryEncoding(num_tokens=index.vocabulary_size())
    
    # apply multi-hot encoding to indices
    # lambda function captures the layer to include them in Keras functional models later
    return lambda feature: encoder(index(feature))

In [68]:
def create_input_data(dataframe):
    all_inputs = []
    encoded_features = []
    
    for header in dataframe.columns:
        # numerical features
        if 'coord' in header or 'already' in header:
            numeric_col = tf.keras.Input(shape=(1,), name=header)
            normalization_layer = get_normalization_layer(header, train_ds)
            encoded_numeric_col = normalization_layer(numeric_col)
            all_inputs.append(numeric_col)
            encoded_features.append(encoded_numeric_col)
        
        # categorical features
        elif 'containment' in header or 'food' in header or 'mid' in header or \
        'strong' in header:
            categorical_col = tf.keras.Input(shape=(1,), name=header, dtype='int64')
            encoding_layer = get_category_encoding_layer(name=header,
                                                        dataset=train_ds,
                                                        dtype='int64')
            encoded_categorical_col = encoding_layer(categorical_col)
            all_inputs.append(categorical_col)
            encoded_features.append(encoded_categorical_col)
            
        elif header == 'input':
            categorical_col = tf.keras.Input(shape=(1,), name='input', dtype='string')
            encoding_layer = get_category_encoding_layer(name='input',
                                                        dataset=train_ds,
                                                        dtype='string')
            encoded_categorical_col = encoding_layer(categorical_col)
            all_inputs.append(categorical_col)
            encoded_features.append(encoded_categorical_col)
            
    return all_inputs, encoded_features

In [69]:
all_inputs, encoded_features = create_input_data(df)

In [70]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)

In [74]:
# use model from tutorial with dense layers

all_features = tf.keras.layers.concatenate(encoded_features)
x = tf.keras.layers.Dense(512, activation="relu")(all_features)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(256, activation="relu")(all_features)
x = tf.keras.layers.Dropout(0.5)(x)
output = tf.keras.layers.Dense(24)(x)

model = tf.keras.Model(all_inputs, output)

In [75]:
model.compile(optimizer='adam',
             loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
             metrics=["accuracy"])

In [81]:
#tf.keras.utils.plot_model(model, show_shapes=True, rankdir='LR')

In [76]:
model.fit(train_ds, epochs=100, validation_data=val_ds, callbacks=[callback])

Epoch 1/100
12/12 [==============================] - 8s 263ms/step - loss: 0.0000e+00 - accuracy: 0.0122 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/100
12/12 [==============================] - 1s 46ms/step - loss: 0.0000e+00 - accuracy: 0.0156 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 3/100
12/12 [==============================] - 1s 43ms/step - loss: 0.0000e+00 - accuracy: 0.0190 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 4/100
12/12 [==============================] - 1s 44ms/step - loss: 0.0000e+00 - accuracy: 0.0347 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 5/100
12/12 [==============================] - 1s 43ms/step - loss: 0.0000e+00 - accuracy: 0.0259 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 6/100
12/12 [==============================] - 1s 44ms/step - loss: 0.0000e+00 - accuracy: 0.0306 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00


In [77]:
loss, accuracy = model.evaluate(test_ds)
print("Accuracy", accuracy)

3/3 [==============================] - 0s 29ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00
Accuracy 0.0


In [28]:
model.save('models/next_obj_classifier_2022-03-23')

2022-03-23 12:41:32.131340: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: models/next_obj_classifier_2022-03-23/assets


INFO:tensorflow:Assets written to: models/next_obj_classifier_2022-03-23/assets


In [31]:
reloaded_model = tf.keras.models.load_model('models/next_obj_classifier_2022-03-23')

In [ ]:
# test prediction for one sample

In [32]:
sample = df.loc[0].drop('target').to_dict()

In [33]:
input_dict = {name: tf.convert_to_tensor([value]) for name, value in sample.items()}

In [34]:
prediction = reloaded_model.predict(input_dict)
prediction = tf.nn.sigmoid(prediction[0])

In [35]:
# get label for prediction

pred_label = labels[np.argmax(prediction)]
pred_label

'p'

In [36]:
def get_prequential_error(dataframe, model, labels, nr_of_sequences):
    errors = [[] for seq in range(0, nr_of_sequences + 1)]
    start_token_count = 0
    sequence_nr = 0
    
    for row in range(0, len(dataframe)): 
        observed_target = dataframe.loc[row, 'target']
        sample = dataframe.loc[row].drop('target').to_dict()
        input_dict = {name: tf.convert_to_tensor([value]) for name, value in 
                          sample.items()}
        predicted_target = model.predict(input_dict)
        predicted_target = tf.nn.sigmoid(predicted_target[0])
            
        pred_label = labels[np.argmax(predicted_target)]
        error = 1 - damerauLevenshtein(pred_label, observed_target)
        errors[sequence_nr].append(error)
        
        if row != 0 and dataframe.loc[row, 'input'] == '<start>':
            start_token_count += 1
        
        if start_token_count > 0:
            sequence_nr += 1
            start_token_count = 0
            
    return errors

In [37]:
errors = get_prequential_error(df, model, labels, nr_of_sequences)

In [38]:
summed_error = [sum(error) for error in errors[:-1]]

In [39]:
np.median(summed_error)

2.0

In [41]:
#with open('results/nn_spatialinfo_prequential_summed_2022-03-23.txt', 'w') as file:
#    file.write(str(summed_error))